<a href="https://polly.elucidata.io/manage/workspaces?action=open_polly_notebook&amp;source=github&amp;path=ElucidataInc%2Fpolly-python%2Fblob%2Fmain%2FAnalyse%2Fmerge_gct_files_using_cmapPy.ipynb&amp;kernel=elucidata%2FPollyglot&amp;machine=large" target="_parent"><img alt="Open in Polly" src="https://elucidatainc.github.io/PublicAssets/open_polly.svg"/></a>


# Welcome to Pollyglot Notebook to get started with merging gct files for any downstream analysis

## Here we have taken the case of Transcriptomics data from TCGA where disease is Breast Neoplasms.

In [11]:
# please do not modify
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [8]:
!sudo pip3 install polly-python # to install polly-python

Looking in indexes: https://pypi.org/simple, http://54.245.179.143:80/
     |████████████████████████████████| 8.6MB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 133kB 91.8MB/s eta 0:00:01
     |████████████████████████████████| 153kB 96.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 47.3MB/s eta 0:00:01
     |████████████████████████████████| 512kB 91.2MB/s eta 0:00:01
     |████████████████████████████████| 143kB 91.7MB/s eta 0:00:01
     |████████████████████████████████| 184kB 91.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 47.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 42.7MB/s eta 0:00:01
     |████████████████████████████████| 153kB 88.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 54.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 52.8MB/s eta 0:00:01
     |████████████████████████████████| 4.0MB 40.1MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-cp36-none-

    Uninstalling boto3-1.17.4:
      Successfully uninstalled boto3-1.17.4
  Found existing installation: pytz 2019.3
    Uninstalling pytz-2019.3:
      Successfully uninstalled pytz-2019.3
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
!sudo pip3 install joblib # to install joblib

Looking in indexes: https://pypi.org/simple, http://54.245.179.143:80/
     |████████████████████████████████| 307kB 7.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
restartkernel() #Pause for a few seconds before the kernel is refreshed

In [3]:
from joblib import Parallel, delayed
import os
import pandas as pd
from cmapPy.pandasGEXpress.parse import parse
from cmapPy.pandasGEXpress import GCToo
from cmapPy.pandasGEXpress import write_gct
from cmapPy.pandasGEXpress.concat import assemble_common_meta
from cmapPy.pandasGEXpress.concat import assemble_data

from polly.auth import Polly
from polly.omixatlas import OmixAtlas
from polly.workspaces import Workspaces

In [4]:
AUTH_TOKEN=(os.environ['POLLY_REFRESH_TOKEN']) # Obtain authentication tokens
Polly.auth(AUTH_TOKEN)

In [5]:
omixatlas = OmixAtlas()

In [6]:
### Dowmload the data parallely

def download_dataset(repo, id):
    """
    Downloads a single dataset with given repo_id and dataset_id
    """
    repo_id = repo
    dataset_id = id
    file_name = f"{dataset_id}.gct"
    data = omixatlas.download_data(repo_id, dataset_id)
    url = data.get('data').get('attributes').get('download_url')
    status = os.system(f"wget -O '{file_name}' '{url}'")
    if status == 0:
        print("Downloaded data successfully")
    else:
        raise Exception("Download not successful")
        
### Read gcts parallely

def read_gcts(dataset_ids):
    gct_files = [dataset_id+'.gct' for dataset_id in dataset_ids]
    results_gct = Parallel(n_jobs=4)(delayed(parse)(gct_file) for gct_file in gct_files)
    return results_gct

In [7]:
query = """SELECT dataset_id, description, disease, tissue, data_type FROM tcga.datasets 
            WHERE data_type ='Transcriptomics' AND disease LIKE '%Breast Neoplasms%'"""
results=omixatlas.query_metadata(query, query_api_version = "v2")
results

Query execution succeeded (time taken: 1.46 seconds, data scanned: 0.322 MB)
Fetched 754 rows


,dataset_id,description,disease,tissue,data_type
0,BRCA_RNASeq_TCGA-EW-A1PD-01A-11R-A144-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
1,BRCA_RNASeq_TCGA-EW-A1P4-01A-21R-A144-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
2,BRCA_RNASeq_TCGA-A7-A0D9-01A-31R-A056-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
3,BRCA_RNASeq_TCGA-EW-A1P8-01A-11R-A144-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
4,BRCA_RNASeq_TCGA-E2-A14T-01A-11R-A115-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
...,...,...,...,...,...
749,BRCA_RNASeq_TCGA-5L-AAT0-01A-12R-A41B-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
750,BRCA_RNASeq_TCGA-A7-A3J1-01A-11R-A213-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
751,BRCA_RNASeq_TCGA-A8-A0A1-01A-11R-A00Z-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics
752,BRCA_RNASeq_TCGA-A2-A4RX-01A-11R-A266-07,Breast invasive carcinoma RNASeq data for TCGA...,['Breast Neoplasms'],['breast'],Transcriptomics


In [13]:
dataset_ids = results["dataset_id"]
dataset_ids = dataset_ids.tolist()

In [14]:
Parallel(n_jobs=20)(delayed(download_dataset)('tcga', i) for i in dataset_ids)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
results_gct = read_gcts(dataset_ids)

In [29]:
### Merge gcts

gct_files = [dataset_id+'.gct' for dataset_id in dataset_ids]
All_Metadata = assemble_common_meta([i.col_metadata_df for i in results_gct], 
                                    fields_to_remove = [], sources = gct_files, 
                                    remove_all_metadata_fields = False, error_report_file = 'errors')

All_data = assemble_data([i.data_df for i in results_gct], concat_direction = 'horiz')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
All_Metadata

chd,barcode,bcr_patient_barcode,curated_age_unit,curated_max_age,curated_min_age,definition,disease_type,is_ffpe,kw_curated_cell_line,kw_curated_cell_type,kw_curated_disease,kw_curated_drug,kw_curated_genetic_mod_type,kw_curated_modified_gene,kw_curated_tissue,name,oct_embedded,patient,primary_site,project_id,releasable,released,sample,sample_id,sample_submitter_id,sample_type,sample_type_id,shortLetterCode,state,submitter_id,tissue_type
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AAAU-01A,Years,55.0,55.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,Goserelin,none,none,breast,Breast Invasive Carcinoma,true,TCGA-3C-AAAU,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-3C-AAAU-01A,197b76a1-a09a-4659-83aa-2c14fd1023a9,TCGA-3C-AAAU-01A,Primary Tumor,1,TP,released,TCGA-3C-AAAU,Not Reported
TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALI-01A,Years,50.0,50.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,Poly E,none,none,breast,Breast Invasive Carcinoma,true,TCGA-3C-AALI,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-3C-AALI-01A,0050d7c9-ece9-4b6c-8023-1ff2efcb3c9c,TCGA-3C-AALI-01A,Primary Tumor,1,TP,released,TCGA-3C-AALI,Not Reported
TCGA-3C-AALK-01A-11R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-3C-AALK-01A,Years,52.0,52.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,Trastuzumab,none,none,breast,Breast Invasive Carcinoma,true,TCGA-3C-AALK,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-3C-AALK-01A,2a532346-0008-4af8-99bc-d709585da1d6,TCGA-3C-AALK-01A,Primary Tumor,1,TP,released,TCGA-3C-AALK,Not Reported
TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-4H-AAAK-01A,Years,50.0,50.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,"Fluorouracil,cyclophosphamide hydrate,doxorubi...",none,none,breast,Breast Invasive Carcinoma,true,TCGA-4H-AAAK,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-4H-AAAK-01A,685b5df4-0620-40e2-aa09-0dd736bb5e2b,TCGA-4H-AAAK-01A,Primary Tumor,1,TP,released,TCGA-4H-AAAK,Not Reported
TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5L-AAT0-01A,Years,42.0,42.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,tamoxifen,none,none,breast,Breast Invasive Carcinoma,false,TCGA-5L-AAT0,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-5L-AAT0-01A,8434f4a2-0ead-40f6-9b63-9c958a3fb812,TCGA-5L-AAT0-01A,Primary Tumor,1,TP,released,TCGA-5L-AAT0,Not Reported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB41-01A,Years,55.0,55.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,"anastrozole,docetaxel anhydrous,tamoxifen,Cycl...",none,none,breast,Breast Invasive Carcinoma,false,TCGA-WT-AB41,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-WT-AB41-01A,0de093d5-c752-4e93-a783-ec262b3a3780,TCGA-WT-AB41-01A,Primary Tumor,1,TP,released,TCGA-WT-AB41,Not Reported
TCGA-WT-AB44-01A-11R-A41B-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-WT-AB44-01A,Years,77.0,77.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,anastrozole,none,none,breast,Breast Invasive Carcinoma,false,TCGA-WT-AB44,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-WT-AB44-01A,081947cf-1635-4a57-9903-0c405ead6d84,TCGA-WT-AB44-01A,Primary Tumor,1,TP,released,TCGA-WT-AB44,Not Reported
TCGA-XX-A89A-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-XX-A89A-01A,Years,68.0,68.0,Primary solid Tumor,"c(""Ductal and Lobular Neoplasms"", ""Complex Epi...",FALSE,none,none,Breast Neoplasms,none,none,none,breast,Breast Invasive Carcinoma,false,TCGA-XX-A89A,Breast,TCGA-BRCA,TRUE,TRUE,TCGA-XX-A89A-01A,1422b157-eec7-460e-96c8-9121847d283b,TCGA-X

In [34]:
All_data

cid,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5T-A9QA-01A-11R-A41B-07,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SD-01A-11R-A115-07,TCGA-A1-A0SE-01A-11R-A084-07,TCGA-A1-A0SF-01A-11R-A144-07,TCGA-A1-A0SG-01A-11R-A144-07,TCGA-A1-A0SJ-01A-11R-A084-07,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A1-A0SN-01A-11R-A144-07,TCGA-A1-A0SO-01A-22R-A084-07,TCGA-A1-A0SP-01A-11R-A084-07,TCGA-A1-A0SQ-01A-21R-A144-07,TCGA-A2-A04N-01A-11R-A115-07,TCGA-A2-A04P-01A-31R-A034-07,TCGA-A2-A04Q-01A-21R-A034-07,TCGA-A2-A04R-01A-41R-A109-07,TCGA-A2-A04U-01A-11R-A115-07,TCGA-A2-A04V-01A-21R-A034-07,TCGA-A2-A04W-01A-31R-A115-07,TCGA-A2-A04X-01A-21R-A034-07,TCGA-A2-A04Y-01A-21R-A034-07,TCGA-A2-A0CK-01A-11R-A22K-07,TCGA-A2-A0CM-01A-31R-A034-07,TCGA-A2-A0CO-01A-13R-A22K-07,TCGA-A2-A0CP-01A-11R-A034-07,TCGA-A2-A0CQ-01A-21R-A034-07,TCGA-A2-A0CS-01A-11R-A115-07,TCGA-A2-A0CT-01A-31R-A056-07,TCGA-A2-A0CU-01A-12R-A034-07,TCGA-A2-A0CV-01A-31R-A115-07,TCGA-A2-A0CW-01A-21R-A115-07,TCGA-A2-A0CX-01A-21R-A00Z-07,TCGA-A2-A0CY-01A-12R-A034-07,TCGA-A2-A0CZ-01A-11R-A034-07,TCGA-A2-A0D0-01A-11R-A00Z-07,...,TCGA-JL-A3YW-01A-12R-A239-07,TCGA-JL-A3YX-01A-11R-A22U-07,TCGA-LD-A74U-01A-13R-A33J-07,TCGA-LL-A441-01A-11R-A24H-07,TCGA-LL-A442-01A-11R-A24H-07,TCGA-LL-A5YN-01A-11R-A28M-07,TCGA-LL-A5YO-01A-21R-A28M-07,TCGA-LL-A6FP-01A-11R-A31O-07,TCGA-LL-A6FR-01A-12R-A31O-07,TCGA-LL-A73Y-01A-11R-A33J-07,TCGA-LL-A7T0-01A-31R-A352-07,TCGA-LL-A8F5-01A-11R-A36F-07,TCGA-MS-A51U-01A-31R-A266-07,TCGA-OK-A5Q2-01A-11R-A27Q-07,TCGA-OL-A5D7-01A-11R-A27Q-07,TCGA-OL-A5D8-01A-11R-A27Q-07,TCGA-OL-A5RV-01A-12R-A28M-07,TCGA-OL-A5RW-01A-11R-A28M-07,TCGA-OL-A5RX-01A-11R-A28M-07,TCGA-OL-A66H-01A-11R-A29R-07,TCGA-OL-A66K-01A-11R-A29R-07,TCGA-OL-A66L-01A-12R-A31O-07,TCGA-OL-A66N-01A-12R-A31O-07,TCGA-OL-A6VO-01A-12R-A33J-07,TCGA-OL-A6VQ-01A-12R-A41B-07,TCGA-OL-A6VR-01A-32R-A33J-07,TCGA-PE-A5DE-01A-11R-A27Q-07,TCGA-PL-A8LZ-01A-31R-A36F-07,TCGA-S3-A6ZF-01A-32R-A32P-07,TCGA-S3-A6ZG-01A-22R-A32P-07,TCGA-S3-A6ZH-01A-22R-A32P-07,TCGA-S3-AA0Z-01A-11R-A41B-07,TCGA-S3-AA12-01A-11R-A41B-07,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07
rid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A1BG,0.344500,0.359700,0.167100,0.235400,0.150100,0.1512,0.0701,0.164700,0.118100,0.154300,0.106800,0.214200,0.182700,0.062300,0.219500,0.050000,0.599000,0.174600,0.060000,0.077600,0.207200,0.186100,0.135600,0.080500,0.042100,0.158100,0.249000,0.134500,0.253900,0.130300,0.206800,0.261800,0.196200,0.143600,0.164800,0.121200,0.059900,0.107200,0.174300,0.013500,...,0.139300,0.267100,0.215000,0.333800,0.233500,0.290100,0.297300,0.201700,0.089300,0.301700,0.197800,0.264600,0.435800,0.106800,0.138200,0.237400,0.252800,0.136100,0.252900,0.030400,0.428400,0.231000,0.147200,0.0605,0.287800,0.307700,0.219400,0.087300,0.234900,0.264800,0.239700,0.4314,0.207500,0.025000,0.274600,0.051400,0.206800,0.210500,0.106500,0.203900
A1BG-AS1,1.519400,2.635600,1.098500,1.563100,1.189200,0.8675,0.5675,1.342700,1.524600,1.578100,1.427500,1.018100,1.106200,0.525900,2.371500,0.376700,3.667600,1.431900,0.403700,1.038500,1.265800,0.467200,1.066400,0.584800,0.548600,1.543200,1.239800,0.699400,1.498200,1.366800,0.736100,1.448300,1.451700,0.888500,0.882800,1.166100,0.718500,1.449200,1.080800,0.077200,...,1.198600,1.464900,1.903800,1.814700,1.262500,1.852000,1.070800,2.002400,0.315900,1.233900,0.891700,0.624600,2.940500,0.675300,0.768100,1.615700,0.771400,0.905800,1.566600,0.449900,2.150500,0.900500,1.511300,0.8681,2.297300,1.875600,2.194500,0.672700,2.063300,1.717700,1.020200,1.8326,1.590300,0.274800,0.934100,0.147500,0.890100,1.403600,1.031200,1.589600
A1CF,0.001800,0.040900,0.007700,0.004300,0.000000,0.0961,0.0021,0.005000,0.000000,0.004200,0.000000,0.005300,0.014100,0.000000,0.004400,0.003